## Attention

複数のデータの中から重要なデータに注目する仕組み。  
Attentionを用いることで、decoderが、encoderが出力した情報の中の重要な情報に注目するようになる。

前章で作成した翻訳モデルは、入力文をencoderによって固定長のベクトルに変換し、それをdecoderに渡すことで、入力文に基づいた出力文を生成した。  
encoderがRNNであるとき、encoderは各時間で隠れ状態を出力する。この中から最後の時間の隠れ状態のみをdecoderに渡していたものがこれまでのseq2seqである。

このとき、encoderが出力する全ての隠れ状態を利用したいと考える。その方が入力文の多くの情報を参照でき、より適切な出力が得られそうだ。Attentionはそれを実現する。

In [17]:
from typing import List
import random

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')


---

## Attention機構

Attention機構について詳しく見ていく。

といっても、そんなに難しいことはない。  
複数のデータがあった時に、各データに重要度を割り当てるだけである。これは重みと呼ばれ$w_i$で表す。重みの総和は1とする。正規化前の重みはスコアと呼んだりする。

重要度は別の何らかのデータに基づいて設定される。一般的に、そのデータとの内積を取ることが多い。

3つの隠れ状態$h_i$があるとする。

In [43]:
batch_size, hidden_size = 3, 5
hs = torch.randn(batch_size, hidden_size)
hs

tensor([[-0.1926,  0.7712, -1.2606,  0.1052, -0.3315],
        [-1.6132,  0.5774, -1.1709, -0.2905, -0.2942],
        [-0.4514,  1.4576,  0.9403,  0.1590, -0.2817]])

隠れ状態$h_i$と同じサイズの適当なデータ$x$があるとする。

In [44]:
x = torch.randn(5)
x

tensor([-0.0412,  0.1183,  0.6055, -0.6576,  0.1853])

$x$と全ての隠れ状態$h_i$で内積を取り、softmaxで正規化する。

In [46]:
scores = hs @ x
weights = F.softmax(scores, dim=-1)
weights

tensor([0.1544, 0.2206, 0.6250])

この3つが重みで、$x$にとっての$h_i$の重要度を表す。

この重みで$h_i$の重み付き和をとることで、$x$にとっての重要度を反映したただ1つの隠れ状態を得ることが出来る。

In [47]:
h = weights @ hs
h

tensor([-0.6677,  1.1575,  0.1347,  0.0515, -0.2921])

seq2seqのdecoderでは、このattentionを利用して、encoderが出力した全ての隠れ状態を参照する。  

encoderの隠れ状態はencoderへの入力数によって変わる。可変長のデータをモデル内で扱うことは困難とされている。  
そこで、attentionを用いて1つの固定長のベクトルに変換する。decoderの演算時に、decoderのRNNから出力された隠れ状態を用いてencoderの隠れ状態の重要度を計算し、重み付き和をとる。こうすることで、encoderの隠れ状態から注目すべき重要な情報を都合よく抽出した固定長のベクトルを得ることが出来る。後はそれを以降の層に渡すだけ。

なお、重みが正しく着目すべき点を表すかは、学習させてみないと分からない。  
この目的も、学習前の段階では期待に過ぎない。この仕組みを取り入れて学習させれば、次第に適切な重みが出力されるようになり、適切な出力が得られるようになるだろう。そうだといいな、ってだけ。

また、重みを求める関数が内積でないといけない理由はない。2つのベクトルからスカラーを得る関数であれば何でもよい。  
内積は類似度を測れ、類似度が高いものに着目するという意味では適切に見えるが、そもそも内積をとるベクトルは入力されてから幾度の変換を経て得られたもので、それらの類似度は意味を持たない。  
重みを求める関数を内積として学習を進めれば、重要度が高くなるべきタイミングでその2つのベクトルが似るように学習される、というだけ。

ただ実際はほとんどの場合で内積が使われる。それは内積という計算がシンプルだからってだけだと思う。

### Attention層

decoderの中で、attentionによって都合のいい隠れ状態を出力する部分は1つの層として見られる。  
実装してみよう。

まずはシンプルなものから。  
単一の時間で機能するものを作る。バッチサイズも1。

In [49]:
class AttentionLayer(nn.Module):
    def forward(self, x, hs):
        """
        x: (hidden_size)
        hs: (seq_len, hidden_size)
        """
        attention_score = hs @ x # (seq_len,)
        weights = F.softmax(attention_score, dim=-1) # (seq_len,)
        y = weights @ hs # (hidden_size,)
        return y

In [51]:
hs = torch.randn(batch_size, hidden_size)
x = torch.randn(hidden_size)

attention = AttentionLayer()
h = attention(x, hs)
h

tensor([ 0.8188,  1.4933, -0.5871, -0.7322, -1.0259])

次はより実践的なものを作る。  
バッチサイズを考慮した上で全ての時間を処理する。

In [53]:
class AttentionLayer(nn.Module):
    def forward(self, x, hs, pad_positions=None):
        """
        x: (batch_size, seq_len_dec, hidden_size)
        hs: (batch_size, seq_len_enc, hidden_size)
        pad_positions: (batch_size, seq_len_enc, hidden_size)
        """
        seq_len_dec = x.shape[1]
        attention_score = torch.matmul(x, hs.transpose(1, 2))
            # (batch_size, seq_len(dec), seq_len(enc))
        weights = F.softmax(attention_score, dim=-1)
        y = [
            (hs * weights[:, i].unsqueeze(-1)).sum(dim=1) \
                for i in range(seq_len_dec)
        ] # (seq_len(dec), batch_size, hidden_size)
        y = torch.stack(y, dim=1) # (batch_size, seq_len(dec), hidden_size)
        return y

これもうちょい綺麗に実装する方法ないのかな。

In [54]:
batch_size, seq_len_dec, seq_len_enc, hidden_size = 3, 5, 7, 11
x = torch.randn(batch_size, seq_len_dec, hidden_size)
hs = torch.randn(batch_size, seq_len_enc, hidden_size)

attention = AttentionLayer()
h = attention(x, hs)
h.shape

torch.Size([3, 5, 11])


---

## 言語モデル

### 学習データ

In [5]:
textfile_ja = 'data/kyoto_ja_10000.txt'
textfile_en = 'data/kyoto_en_10000.txt'

with open(textfile_en) as f:
    data_en = f.readlines()

with open(textfile_ja) as f:
    data_ja = f.readlines()

n_data = len(data_en)
print('num of data:', n_data)

num of data: 10000


In [6]:
tokenizer_prefix_ja = 'models/tokenizer_kyoto_ja_10000'
tokenizer_prefix_en = 'models/tokenizer_kyoto_en_10000'
sp_ja = spm.SentencePieceProcessor(f'{tokenizer_prefix_ja}.model')
sp_en = spm.SentencePieceProcessor(f'{tokenizer_prefix_en}.model')
n_vocab_ja = len(sp_ja)
n_vocab_en = len(sp_en)
pad_id = sp_ja.pad_id()
print('num of vocabrary (ja):', n_vocab_ja)
print('num of vocabrary (en):', n_vocab_en)

num of vocabrary (ja): 8000
num of vocabrary (en): 8000


In [7]:
data_ids_ja = sp_ja.encode(data_ja)
data_ids_en = sp_en.encode(data_en)

In [8]:
bos_id = sp_ja.bos_id()
eos_id = sp_ja.eos_id()
for ids_ja, ids_en in zip(data_ids_ja, data_ids_en):
    ids_en.insert(0, bos_id)
    ids_ja.append(eos_id)
    ids_en.append(eos_id)

In [10]:
class TextDataset(Dataset):
    def __init__(self, data_ids_ja, data_ids_en):
        self.data_ja = [torch.tensor(ids) for ids in data_ids_ja]
        self.data_en = [torch.tensor(ids) for ids in data_ids_en]
        self.n_data = len(self.data_ja)

    def __getitem__(self, idx):
        ja = self.data_ja[idx]
        en = self.data_en[idx]
        x_enc = ja # encoderへの入力
        x_dec = en[:-1] # decoderへの入力
        y_dec = en[1:] # decoderの出力
        return x_enc, x_dec, y_dec

    def __len__(self):
        return self.n_data

def collate_fn(batch): # padding
    x_enc, x_dec, y_dec= zip(*batch)
    x_enc = pad_sequence(x_enc, batch_first=True, padding_value=pad_id)
    x_dec = pad_sequence(x_dec, batch_first=True, padding_value=pad_id)
    y_dec = pad_sequence(y_dec, batch_first=True, padding_value=pad_id)
    return x_enc, x_dec, y_dec

batch_size = 1
batch_size = 64
dataset = TextDataset(data_ids_ja, data_ids_en)
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn
)

### モデル

In [11]:
class Encoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        """
        x: (batch_size, seq_len)
        """
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        hs, h = self.rnn(x)
            # hs: (batch_size, seq_len, hidden_size)
            # h: (1, batch_size, hidden_size)
        hs = self.fc(hs) # (batch_size, seq_len, hidden_size)
        return hs, h

In [12]:
class Decoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.attention = AttentionLayer()
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, h, hs):
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        hs_dec, h = self.rnn(x, h)
            # hs_dec: (batch_size, seq_len, hidden_size)
            # h: (1, batch_size, hidden_size)
        y = self.attention(hs_dec, hs) # (batch_size, seq_len, hidden_size)
        y = self.fc(y) # (batch_size, seq_len, n_vocab)
        return y, h

In [13]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x_enc, x_dec):
        hs, h = self.encoder(x_enc)
        y, _ = self.decoder(x_dec, h, hs)
        return y

In [14]:
def train(model, optimizer, criterion, n_epochs, prog_unit=1):
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        for x_enc, x_dec, y_dec in dataloader:
            optimizer.zero_grad()
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)

            y_pred = model(x_enc, x_dec)
            loss = criterion(y_pred.reshape(-1, n_vocab_ja), y_dec.ravel())
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

In [15]:
hidden_size, embed_size = 1024, 1024
encoder = Encoder(n_vocab_ja, embed_size, hidden_size)
decoder = Decoder(n_vocab_en, embed_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

In [16]:
train(model, optimizer, criterion, n_epochs=100, prog_unit=10)

   1-10/100: ######################################## 100% [00:16:17.81] loss: 1.73452 
  11-20/100: ####################                      50% [00:09:13.56] loss: 1.54642 

KeyboardInterrupt: 

In [34]:
unk_id = sp_en.unk_id() # UNKのID
def token_sampling(y: List[float]) -> int:
    """モデルの出力から単語をサンプリングする"""
    y[unk_id] = -torch.inf
    probs = F.softmax(y, dim=-1)
    token, = random.choices(range(n_vocab_en), weights=probs)
    return token


bos_id = sp_en.bos_id()
eos_id = sp_en.eos_id()
@torch.no_grad()
def translate(
    model: nn.Module,
    in_text: str, # 入力文（日本語）
    max_len: int = 100, # 出力のトークン数の上限
    decisive: bool = True, # サンプリングを決定的にするか
) -> str:
    model.eval()
    in_ids = sp_ja.encode(in_text)
    in_ids = torch.tensor(in_ids + [eos_id], device=device).unsqueeze(0)

    hs, h = model.encoder(in_ids)
    next_token = bos_id

    token_ids = []
    for _ in range(max_len):
        x = torch.tensor([[next_token]], device=device)
        y, h = model.decoder(x, h, hs)
        y = y[0]
        if decisive:
            next_token = y.argmax().item()
        else:
            next_token = token_sampling(y)
        token_ids.append(next_token)
        if next_token == eos_id:
            break
    sentence = sp_en.decode(token_ids)
    return sentence

In [35]:
n = 5
for x, t in zip(data_ja[:n], data_en[:n]):
    print('input:', x)
    print('output:', translate(model, x))
    print('answer:', t)
    print()

input: 駅情報

output: 
answer: Information


input: 三条京阪駅（さんじょうけいはんえき）は、京都市東山区にある、京都市営地下鉄東西線の鉄道駅。

output: cho Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto Kyoto
answer: Located in the Higashiyama Ward of Kyoto City, Sanjyo-Keihan Station is a stop on the Tozai Line, a Kyoto Municipal Subway Line.


input: 駅番号はT11。

output: The station station station station station station station station station station station station station station station station s